In [6]:
##########################################################
## VHDL Library Generator - AMD FPGA                    ##
##########################################################
##          Product_VHDL_WeightPKG-Coef                 ##
##          ---------------------------                 ##
##                                       24-04-2025     ##
## Program that generates the weight-matrix files       ##
## - code for AMD FPGA                                  ##
## automated VHDL code generation -> BINDER.vhd         ##
##                                                      ##
## Version 1 - no corrections - add min max             ##
##                                - add sumpos sumneg   ##
##                                                      ##          
##########################################################
## -> Correction of weight quantization                 ##
##     adjust - MybinaryVal :  (one among all)          ##
##                 -> 256, 128, 64, 32, 16, 8, 4, 2, 1  ##
##                                                      ##
##########################################################
#################################################
## quantization of the SRC weight matrix       ##
#################################################
const MybinaryVal = 19

# for reading and processing the JSON file
using JSON3

# for reading and processing the NPZ file
using NPZ

# used if needed for displaying results
using Printf


In [7]:
##########################################################
## Creation of JSON3 file-reading methods               ##
##########################################################
## program to set up a library                          ##
## for reading the different parameters encoded in a    ##
## JSON3 file                                           ##
##                                                      ##
##                Pascal Harmeling 2023 - ULiège        ##
##                                                      ##
##########################################################

#################################################
## Function use JSON file input - read values  ##
#################################################
json_string = read("model/config.json", String)

FileJson = JSON3.read(json_string)

# principle for accessing elements of the data structure.
const cell_config_alphas=FileJson.cell_config.alphas
const cell_config_recurrence=FileJson.cell_config.recurrence
const cell_config_bias_max=FileJson.cell_config.bias_max
const cell_config_activation=FileJson.cell_config.activation
const cell_config_type=FileJson.cell_config.type
const cell_config_num_integrators_per_neuron=FileJson.cell_config.num_integrators_per_neuron

const dataset_config_encoding=FileJson.dataset_config.encoding
const dataset_config_seq_length::Int16=FileJson.dataset_config.seq_length
const dataset_config_prefix_length=FileJson.dataset_config.prefix_length
const dataset_config_suffix_length=FileJson.dataset_config.suffix_length
const dataset_config_rate_gain=FileJson.dataset_config.rate_gain
const dataset_config_name=FileJson.dataset_config.name
    
const input_size=FileJson.input_size
const hidden_size=FileJson.hidden_size
const hidden_layers=FileJson.hidden_layers
const output_size=FileJson.output_size
const readout_input_factor=FileJson.readout_input_factor


10

In [8]:
##########################################################
## Creation of NPY file-reading methods                 ##
##########################################################
## program to set up a library                           ##
## for reading the different parameters encoded in an    ##
## NPY file                                              ##
##                                                      ##
##                Pascal Harmeling 2023 - ULiège        ##
##                                                      ##
##########################################################

#################################################
## Function use NPY file input - read values   ##
#################################################

vars=npzread("model/arrays.npz")

# retrieval of the different parameters and initial values - vectorization (vec()) if needed
const bias=get(vars,"l0_bias", 0)
const forward_weights=get(vars,"l0_forward_weights", 0)

const Koeff= (abs(maximum(forward_weights)) > abs(minimum(forward_weights))) ? MybinaryVal/abs(maximum(forward_weights)) : MybinaryVal/abs(minimum(forward_weights))
const forward_weightsp=round.(Int16,(get(vars,"l0_forward_weights", 0).*Koeff))


const initial_h=get(vars,"l0_initial_h", 0)
const initial_hs=get(vars,"l0_initial_hs", 0)
const initial_i=vec(get(vars,"l0_initial_i", 0)[1,:,1])   # VECTORIZATION 
const r=get(vars,"l0_r", 0)
const rho=get(vars,"l0_rho", 0)
const rs=get(vars,"l0_rs", 0)
const zs_dep=vec(get(vars,"l0_zs_dep", 0)[1,:])
const zs_hyp=vec(get(vars,"l0_zs_hyp", 0)[1,:])
const readout=get(vars,"readout", 0)
const readoutp=round.(Int8, readout.*10)


10×100 Matrix{Int8}:
 10  -1  -1  -1  -1  -1  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  10  -1  -1  -1  -1  -1  -1  -1     10  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  10  -1  -1  -1  -1  -1  -1     -1  10  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1  10  -1  -1  -1  -1  -1     -1  -1  10  -1  -1  -1  -1  -1  -1
 -1  -1  -1  -1  10  -1  -1  -1  -1     -1  -1  -1  10  -1  -1  -1  -1  -1
 -1  -1  -1  -1  -1  10  -1  -1  -1  …  -1  -1  -1  -1  10  -1  -1  -1  -1
 -1  -1  -1  -1  -1  -1  10  -1  -1     -1  -1  -1  -1  -1  10  -1  -1  -1
 -1  -1  -1  -1  -1  -1  -1  10  -1     -1  -1  -1  -1  -1  -1  10  -1  -1
 -1  -1  -1  -1  -1  -1  -1  -1  10     -1  -1  -1  -1  -1  -1  -1  10  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  10

In [9]:
# read the matrix, convert it, and write it to the output file - VHD
# SRCOUT GENERATION
level = 1;
Slevel = @sprintf("%02d", level)

filename = "WeightMatrix01_pkg(+-" * lpad(string(MybinaryVal), 3, '0') * ").vhd"
file=open(filename, "w")  # !!!-> in automatic mode, make sure to generate the name correctly !!!
    
write(file, string("----------------------------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("---  producted Package by julia - Weight matrix\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("-- Pascal Harmeling 2025\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------\n"))
write(file, string("library ieee;\n"))
write(file, string("use ieee.std_logic_1164.all;\n"))
write(file, string("use ieee.numeric_std.all;\n"))
write(file, string("\n"))
write(file, string("package WeightMatrix$Slevel"," is\n"))
write(file, string("	-- Taille de la matrice\n"))
write(file, string("    constant ML$Slevel","X : natural := 100;\n"))
write(file, string("    constant ML$Slevel","Y : natural := 784;\n"))
write(file, string("\n"))
write(file, string("    -- Déclaration de la matrice de poids\n"))
write(file, string("	type framebuffer_ML$Slevel is array (0 to ML$Slevel","X-1, 0 to ML$Slevel","Y-1) of integer;\n"))
write(file, string("\n"))
write(file, string("    constant ML$Slevel : framebuffer_ML$Slevel :=(\n"))

min=0
max=0
cmp=0
tmpsumspos=0
tmpsumsneg=0
sumspos=0
sumsneg=0

for x=0:1:99
    tmpsumspos=0
    tmpsumsneg=0
    Sx = @sprintf("%3d", x)
    write(file, string("		   $Sx => ("))
    for y=0:1:783
        cmp=Koeff*forward_weights[x+1,y+1]
        val = @sprintf("%4d", round(Int16,cmp))
        if (min>cmp) min=cmp end #chercher le min
        if (max<cmp) max=cmp end #chercher le max
        if (cmp>0) tmpsumspos = tmpsumspos + cmp end
        if (cmp<0) tmpsumsneg = tmpsumsneg + cmp end
        if y<783
            write(file, string(" $val, "))
        else
            write(file, string(" $val"))
        end
     end
     if x<99
         write(file, string("),\n"))
    else 
         write(file, string(")\n"))
    end
    if (tmpsumspos>sumspos) sumspos=tmpsumspos end
    if (tmpsumsneg<sumsneg) sumsneg=tmpsumsneg end
end 
write(file, string("        );\n"))
write(file, string("\n"))
write(file, string(" ---------------------------------INFO-\n"))
write(file, string(" -- COEF =$Koeff\n"))
write(file, string("\n"))
write(file, string(" -- MIN =$min\n"))
write(file, string(" -- MAX =$max\n"))
write(file, string("\n"))
write(file, string(" -- SUMMIN =$sumsneg\n"))
write(file, string(" -- SUMMAX =$sumspos\n"))
write(file, string(" ---------------------------------INFO-\n"))

# Write get-set function  
write(file, string("\n")) 
write(file, string("-- Fonction d'accès à un élément de la matrice\n"))
write(file, string("  function get_ML$Slevel(\n"))
write(file, string("    framebuffer : framebuffer_ML$Slevel;\n"))
write(file, string("        y           : integer;\n"))
write(file, string("        x           : integer\n"))
write(file, string("    ) return integer;\n"))
write(file, string("\n"))
write(file, string("end WeightMatrix$Slevel",";\n"))
write(file, string("\n"))
write(file, string("\n"))
write(file, string("package body WeightMatrix$Slevel"," is\n"))
write(file, string("  function get_ML$Slevel(\n"))
write(file, string("    framebuffer : framebuffer_ML$Slevel;\n"))
write(file, string("        y           : integer;\n"))
write(file, string("        x           : integer\n"))
write(file, string("    ) return integer is\n"))
write(file, string("    begin\n"))
write(file, string("        return framebuffer(y , x);\n"))
write(file, string("    end function;\n"))
write(file, string("\n"))
write(file, string("end package body WeightMatrix$Slevel",";\n"))

close(file)

In [10]:
# read the matrix, convert it, and write it to the output file - VHD
# GENERATION ROUT
level = 2;
Slevel = @sprintf("%02d", level)

file=open("WeightMatrix02_pkg(bit).vhd", "w")  # !!!-> in automatic mode, make sure to generate the name correctly !!!
write(file, string("----------------------------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("---  producted Package by julia - Weight matrix\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------\n"))
write(file, string("---\n"))
write(file, string("-- Pascal Harmeling 2025\n"))
write(file, string("---\n"))
write(file, string("----------------------------------------------------\n"))
write(file, string("library ieee;\n"))
write(file, string("use ieee.std_logic_1164.all;\n"))
write(file, string("use ieee.numeric_std.all;\n"))
write(file, string("\n"))
write(file, string("package WeightMatrix$Slevel"," is\n"))
write(file, string("	-- Taille de la matrice\n"))
write(file, string("    constant ML$Slevel","X : natural := 10;\n"))
write(file, string("    constant ML$Slevel","Y : natural := 100;\n"))
write(file, string("    constant ML$Slevel","Val0 : integer := -1;\n"))
write(file, string("    constant ML$Slevel","Val1 : integer := 10;\n"))
write(file, string("\n"))
write(file, string("    -- Déclaration de la matrice de poids\n"))
write(file, string("	type framebuffer_ML$Slevel is array (0 to ML$Slevel","X-1, 0 to ML$Slevel","Y-1) of std_logic;\n"))
write(file, string("	type int_array_ML$Slevel is array (0 to ML$Slevel","X-1) of integer range -2000 to 2000;  -- avant -20000 to +20000\n"))
write(file, string("\n"))
write(file, string("    constant ML$Slevel : framebuffer_ML$Slevel :=(\n"))

min=0
max=0
cmp=0
tmpsumspos=0
tmpsumsneg=0
sumspos=0
sumsneg=0

for x=0:1:9
    tmpsumspos=0
    tmpsumsneg=0
    Sx = @sprintf("%3d", x)
    write(file, string("		   $Sx => ("))
    for y=0:1:99
        cmp=readoutp[x+1,y+1]
        if (cmp == -1) cmp = 0
            else cmp = 1
        end
        val = @sprintf("'%1d'", round(Int16,cmp)) #string(round(Int16,cmp))
        if (min>cmp) min=cmp end #chercher le min
        if (max<cmp) max=cmp end #chercher le max
        if (cmp>0) tmpsumspos = tmpsumspos + cmp end
        if (cmp<0) tmpsumsneg = tmpsumsneg + cmp end
        if y<99
            write(file, string(" $val, "))
        else
            write(file, string(" $val"))
        end
     end
     if x<9
         write(file, string("),\n"))
    else 
         write(file, string(")\n"))
    end
    if (tmpsumspos>sumspos) sumspos=tmpsumspos end
    if (tmpsumsneg<sumsneg) sumsneg=tmpsumsneg end
end 
write(file, string("        );\n"))
write(file, string("\n"))
write(file, string(" ---------------------------------INFO-\n"))
write(file, string(" -- MIN =$min\n"))
write(file, string(" -- MAX =$max\n"))
write(file, string("\n"))
write(file, string(" -- SUMMIN =$sumsneg\n"))
write(file, string(" -- SUMMAX =$sumspos\n"))
write(file, string(" ---------------------------------INFO-\n"))

# Write get-set function  
write(file, string("\n")) 
write(file, string("-- Fonction d'accès à un élément de la matrice\n"))
write(file, string("  function get_ML$Slevel(\n"))
write(file, string("    framebuffer : framebuffer_ML$Slevel;\n"))
write(file, string("        y           : integer;\n"))
write(file, string("        x           : integer\n"))
write(file, string("    ) return std_logic;\n"))
write(file, string("\n"))
write(file, string("end WeightMatrix$Slevel",";\n"))
write(file, string("\n"))
write(file, string("\n"))
write(file, string("package body WeightMatrix$Slevel"," is\n"))
write(file, string("  function get_ML$Slevel(\n"))
write(file, string("    framebuffer : framebuffer_ML$Slevel;\n"))
write(file, string("        y           : integer;\n"))
write(file, string("        x           : integer\n"))
write(file, string("    ) return std_logic is\n"))
write(file, string("    begin\n"))
write(file, string("        return framebuffer(y , x);\n"))
write(file, string("    end function;\n"))
write(file, string("\n"))
write(file, string("end package body WeightMatrix$Slevel",";\n"))

close(file)